In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

# Define data transforms
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Load train and test data
train_data = datasets.ImageFolder('/content/drive/MyDrive/Covid19-dataset/train', transform=data_transforms)
test_data = datasets.ImageFolder('/content/drive/MyDrive/Covid19-dataset/test', transform=data_transforms)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

# Load pre-trained ResNet50
model = torchvision.models.resnet50(pretrained=True)

# Freeze all layers except the last fully connected layer and the layers in the final residual block
for param in model.parameters():
    param.requires_grad = False
    
for param in model.layer4.parameters():
    param.requires_grad = True
    
for param in model.fc.parameters():
    param.requires_grad = True
    
# Modify last fully connected layer to match number of classes
model.fc = torch.nn.Linear(2048, 10)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Initialize tensorboard writer
writer = SummaryWriter('logs')

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get inputs and labels
        inputs, labels = data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            writer.add_scalar('training loss', running_loss / 10, epoch * len(train_loader) + i)
            running_loss = 0.0

    # Test the model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))
    writer.add_scalar('test accuracy', 100 * correct / total, epoch)

print('Finished Training')

# Save the trained model
torch.save(model.state_dict(), 'resnet.pth')

# Close tensorboard writer
writer.close()


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 310MB/s]


Accuracy of the network on the test images: 39 %
Accuracy of the network on the test images: 53 %
Accuracy of the network on the test images: 46 %
Accuracy of the network on the test images: 53 %
Accuracy of the network on the test images: 56 %
Accuracy of the network on the test images: 56 %
Accuracy of the network on the test images: 59 %
Accuracy of the network on the test images: 59 %
Accuracy of the network on the test images: 59 %
Accuracy of the network on the test images: 60 %
Finished Training
